In [56]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import threading
import os

In [115]:
resultados_finais = []

def verifica_issn(issn='0100-1574', site_url='', navegador=webdriver.Chrome):
    navegador.get(site_url)

    # Pesquisar o valor
    caixa = navegador.find_element(By.XPATH, r'/html/body/div[4]/div/form/div/div/div[1]/input')
    caixa.send_keys(issn)
    navegador.find_element(By.XPATH, r'/html/body/div[4]/div/form/div/div/input').click()

    # Verificar se existe o campo e se constam 2 ou mais registros
    try:
        valor = navegador.find_element(By.XPATH, r'/html/body/div[4]/div/div[2]/div[1]/div/div[3]/div/div/form/div/div[1]/h2').text
    except:
        try:
            valor = navegador.find_element(By.XPATH, r'/html/body/div[5]/div/div[1]/div/div/div[1]/h3').text
        except:
            valor = 'Registro único'

    return valor

def webscraper(list_issn=list):
    global resultados_finais

    site = 'https://portal.issn.org/'

    # Abrir o navegador
    cService = webdriver.ChromeService(executable_path='C:\chromedriver.exe')
    navegador = webdriver.Chrome(service = cService)
    
    resultados = []
    for issn in list_issn:
        resultado = verifica_issn(issn, site_url=site, navegador=navegador)
        resultados.append(resultado)

    navegador.quit()

    resultado_final = pd.DataFrame({'ISSN':list_issn, 'Resultado':resultados})
    resultados_finais.append(resultado_final)

Gera arquivo com ISSN válidos

In [2]:
base_issn = pd.read_csv('registro_bibliografico_202410290742.csv', delimiter=';')
base_issn = base_issn[base_issn["issn"].str.len() <= 9]
base_issn.shape

Carrega Lista ISSN com Categoria (Brasil ou não)

In [55]:
issn_full = pd.read_excel(r'K:/Download/Bibliodata ISSN/Fisico x Digital - ISSN.xlsx', sheet_name='Bibliodata')

# Filtra apenas registros fora do Brasil
issn_full = issn_full[issn_full['É do Brasil?'] == False]
issn_full.shape

(10798, 5)

In [108]:
# Separa os registros em 6 listas
issn_full_list = issn_full['issn'].to_list()
list1, list2, list3, list4, list5, list6, list7, list8, list9, list10 = np.array_split(issn_full_list, 10)

In [116]:
worker1 = threading.Thread(target=webscraper, args=(list1,))
worker2 = threading.Thread(target=webscraper, args=(list2,))
worker3 = threading.Thread(target=webscraper, args=(list3,))
worker4 = threading.Thread(target=webscraper, args=(list4,))
worker5 = threading.Thread(target=webscraper, args=(list5,))
worker6 = threading.Thread(target=webscraper, args=(list6,))
worker7 = threading.Thread(target=webscraper, args=(list7,))
worker8 = threading.Thread(target=webscraper, args=(list8,))
worker9 = threading.Thread(target=webscraper, args=(list9,))
worker10 = threading.Thread(target=webscraper, args=(list10,))

worker1.start()
worker2.start()
worker3.start()
worker4.start()
worker5.start()
worker6.start()
worker7.start()
worker8.start()
worker9.start()
worker10.start()

worker1.join()
worker2.join()
worker3.join()
worker4.join()
worker5.join()
worker6.join()
worker7.join()
worker8.join()
worker9.join()
worker10.join()

In [121]:
datasets = resultados_finais

issn_full_final = pd.DataFrame()
for c in datasets:
    issn_full_final = pd.concat([issn_full_final, c])

In [123]:
issn_full_final.to_excel('issn_nao_brasil_final.xlsx', index=False)